# Generacion de voto
Correr estos scripts correctamente genera un input del voto cifrado y la ZKP necesaria para hacer la transacción en Sepolia y efectuar el voto demostrando la habilitación para votar y el correcto cifrado del voto sin revelar el token de votación ni el candidato elegido en el voto

## Setup
Estamos suponiendo que estás en un entorno de Linux. Se puede hacer en WSL. No probamos compatibilidad con Mac.

### Para instalar ZoKrates, ejecutar el siguiente comando
```bash
curl -LSfs get.zokrat.es | sh
```
Luego, no olvides hacer
```bash
export PATH=$PATH:/home/"$USER"/.zokrates/bin
```
reemplazando $user por tu usuario 

### Para curvas elípticas
```bash
pip install zokrates_pycrypto
```

También vamos a usar las librerías `secrets` y `hashlib`. Vienen incluidas en las versiones recientes de python.



## Declaración de variables
Completar la siguiente celda con tu token de voto, tokenHash y elección binaria

In [ ]:
# Variables privadas - CAMBIAR POR VALORES REALES
token = [
0x01234567,
0x89abcdef,
0x01234567,
0x89abcdef,
0x01234567,
0x89abcdef,
0x01234567,
0x89abcdef
]

tokenHash = [
    0x89abcdef,
    0x01234567,
    0x89abcdef,
    0x01234567,
    0x89abcdef,
    0x01234567,
    0x889abcdef,
    0x01234567
]

Voto = None # Cambiar por True o False

### Cálculo de entrada para ZoKrates
Vamos a calcular los elementos necesarios para probar que conocemos un token válido y que 

In [ ]:
import secrets
import zokrates_pycrypto
import zokrates_pycrypto.babyjubjub as BJJ


In [ ]:
# Para más adelanta
import secrets
import zokrates_pycrypto.babyjubjub as BJJ
import zokrates_pycrypto
import hashlib

# Definición del Merkle Tree (público)

class TreeNode:
  def __init__(self, data):
    self.data = data
    self.left = None
    self.right = None


# Merkle tree HARDCODEADO para un MerkleInclusion de depth = 2 
# Preimágenes en el archivo secrets.csv

merkleTree = TreeNode([0x6e91d0d2,0x78aa6339,0xc4c452ac,0xf6e51270,0x8aa5bd3b,0xc1e92ee5,0x0815a83d,0x371f8b2e])

ab=TreeNode([0xb8de98ae,0xef23fbc6,0x51d4df0f,0x6aaf91ba,0xf9b1d7c8,0x920368c7,0x37899bc8,0xc2138595])
cd=TreeNode([0x95854de9,0x383e0c5d,0xaf6ff238,0x03254382,0xd868b844,0x4596c4d5,0xe279ef9f,0x6485e91f])
merkleTree.left=ab
merkleTree.right=cd

a = TreeNode([0x8eaa9432,0x25c5e485,0x00500bff,0x9d7f4fd9,0x310d40c7,0x011f961f,0x4196fc2b,0x3bf2302e])
b = TreeNode([0xc047ba93,0xdfeab47c,0x31e66179,0x03392d45,0x861700c7,0xbddc2180,0xe5d629a9,0xabbf3d6c])
c = TreeNode([0xd8adb1aa,0xf32e4863,0xf9d685da,0xd4287608,0x54447dec,0x73ea4ef7,0x4851cc38,0x885903f3])
d = TreeNode([0xf76a00ec,0xe161e995,0x5819f907,0x8bc2a2b2,0x2f8802e7,0xc706875d,0xca40ad66,0x78552242])
ab.left=a
ab.right=b
cd.left=c
cd.right=d


IZQUIERDA = 0
DERECHA = 1
camino = [0,0]
directionSelector = [0,0]
altura = 2
def calcularCamino(camino, directionSelector, root, altura, indice):
    if altura > 0:
      if (indice <= 2**(altura-1)):
        next = root.left
        camino[altura-1] = root.right.data #Camino guarda los que NO calculamos
        directionSelector[altura-1] = IZQUIERDA
      else:
        next = root.right
        camino[altura-1] = root.left.data
        directionSelector[altura-1] = DERECHA


        indice -= 2 ** (altura - 1)# Indice en el subarbol derecho
      return calcularCamino(camino, directionSelector, next, altura-1, indice)
    else:
      return (camino, directionSelector)
    
def indiceDeToken(tokenHash):
  for idx, leaf in enumerate([a,b,c,d]):
    if tokenHash == leaf.data: return idx + 1
  raise Exception("TokenHash doesn't belong to the tree")

calcularCamino(camino, directionSelector, merkleTree, 2, indiceDeToken(tokenHash))

# Con esto ya calculamos todo lo necesario para la demostración de inclusión de nuestro token en el árbol.

# Ahora generamos un valor aleatorio y encriptamos el valor de nuestro voto
r = secrets.randbelow(zokrates_pycrypto.field.field_modulus)

# Valores del generador de la curva BBJ
generator_x = 16540640123574156134436876038791482806971768689494387082833631921987005038935
generator_y = 20819045374670962167435360035096875258406992893633759881276124905556507972311

# Clave pública (cambiaría para otra votación)
publicKey_x = 13430624290144610199046917844800525545438984275839206292223261386245851008921
publicKey_y = 12875888610528699823168293398233054693900374645852792388750630730514979280099

G = BJJ.Point(generator_x, generator_y)

H = BJJ.Point(publicKey_x, publicKey_y)

# Primer componente de nuestro voto encriptado
C1 = G.mult(r)

# Acá entra la variable "Voto" definida previamente, se encripta como el mensaje
mG = G.mult(int(Voto))

rH = H.mult(r)
# Segundo componente de nuestro voto encriptado
C2 = mG.add(rH)

# Con esto ya tenemos todo calculado, ahora solo queda calcular el nullifier
# Para eso vamos a implementar una función de hash equivalente a la que usa la ZKP

def zokrates_sha256_512(a, b):
    if len(a) != 8 or len(b) != 8:
        raise ValueError("Deben ser u32[8]")

    data = b""
    for x in a + b:
        if x < 0 or x >= 2**32:
            raise ValueError("Elemento fuera de u32")
        data += x.to_bytes(4, byteorder="big", signed=False)

    digest = hashlib.sha256(data).digest()

    return [
        int.from_bytes(digest[i*4:(i+1)*4], "big")
        for i in range(8)
    ]

#### Para la creacion del nullifier
 Utilizamos las funciones de ZoKrates para calcular el hash de nuestro Token con una máscara

In [ ]:
mask = [0x01234567,0x89abcdef,0x01234567,0x89abcdef,0x01234567,0x89abcdef,0x01234567,0x89abcdef] # Para que no se sepa de que hoja viene el voto
maskStr = str(mask)

maskStr=maskStr.replace(","," ")
maskStr=maskStr.replace("[","")
maskStr=maskStr.replace("]"," ")
maskStr=maskStr.replace("'","")

tokenStr = str(token)

tokenStr=tokenStr.replace(","," ")
tokenStr=tokenStr.replace("[","")
tokenStr=tokenStr.replace("]"," ")
tokenStr=tokenStr.replace("'","")

print(f"zokrates compute-witness -a {tokenStr} {maskStr}")

Abrir una terminal en la carpeta donde se encuentre `nullifierCalculator.zok` y **ejecutar el comando generado por la celda anterior**, el mismo va a tener este formato:

```bash
zokrates compute-witness -a <parametros generados por el script>
```

Una vez obtenido el resultado, pegar el valor en la celda siguiente para definir la variable "nullifier"

In [ ]:
nullifier = []  #COMPLETAR - Por favor eliminar los nombres, los dos puntos y las comillas
                #dejar solo array de valores

nullifier.reverse() # En witness.json vienen ordenados al reves los valores
print([hex(v) for v in nullifier])

In [ ]:


# Ahora formateamos los resultados para que puedan ser enviados fácilmente a ZoKrates

caminoStr = str(camino)
directionSelectorStr = str(directionSelector)

caminoStr=caminoStr.replace(","," ")
caminoStr=caminoStr.replace("[","")
caminoStr=caminoStr.replace("]"," ")
caminoStr=caminoStr.replace("'","")

directionSelectorStr=directionSelectorStr.replace(","," ")
directionSelectorStr=directionSelectorStr.replace("[","")
directionSelectorStr=directionSelectorStr.replace("]"," ")
directionSelectorStr=directionSelectorStr.replace("'","")


c1_x = str(C1.x)
c1_y = str(C1.y)
c2_x = str(C2.x)
c2_y = str(C2.y)
votoStr = str(int(Voto))


def f(input):
    return ["".join(input[:8]), "0x" + "".join(input[8:16]), "0x" + "".join(input[16:24]), "0x"+ "".join(input[24:32]), "0x" + "".join(input[32:40]), "0x" + "".join(input[40:48]), "0x" + "".join(input[48:56]), "0x"+ "".join(input[56:64])]

f(r)
rStr = str(r)
nullifierStr = str(nullifier)
nullifierStr=nullifierStr.replace(","," ")
nullifierStr=nullifierStr.replace("[","")
nullifierStr=nullifierStr.replace("]"," ")
nullifierStr=nullifierStr.replace("'","")


command = "zokrates compute-witness -a "
print(command + tokenStr + directionSelectorStr + caminoStr + c1_x + " " + c1_y + " " + c2_x + " " + c2_y + " " + votoStr + " " + rStr + " " + nullifierStr)

# Creación de la ZKP en la consola utilizando el input generado

Ejecuta el output generado por la celda en una terminal y eso generará el "witness" necesario para crear la prueba usando zokrates

Para crear la prueba primero descomprimir el archivo .tar.gz utilizando el comando:

```bash
tar -xzvf validVote.tar.gz
```

Luego, abrir una terminal en la carpeta donde se encuentre `merkle` y **ejecutar el comando generado por la celda anterior**, el mismo va a tener este formato:

```bash
zokrates compute-witness -a <parametros generados por el script>
```

Finalmente, ejecutar:

```bash
zokrates generate-proof
```


## Enviar prueba al contrato

Para enviar la prueba al contrato, tenemos que enviarle los parámetros. Vamos a ayudarnos con la interfaz de Remix IDE.

El código del contrato se encuentra dentro de VotingSystem.sol en este mismo directorio y la dirección del contrato es la siguiente 

Dirección del contrato:

    Para presentar este código completar con una dirección del contrato desplegado en la red de sepolia

El contrato recibe los siguientes parámetros:

uint256[2] vote: Los valores G1 y G2 calculados previamente

Proof proof: El resultado de la ZKP generado por el comando anterior

uint32[8] nullifier: El valor del nullifier generado previamente